In [23]:
1+1

2

- [kaggle_url](https://www.kaggle.com/competitions/playground-series-s3e20)


In [24]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [25]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [26]:
def place_as_index(df):
  location_df = df[['latitude', 'longitude']].drop_duplicates().reset_index().drop('index', axis = 1)
  location_df['No'] = range(1, len(location_df.index) + 1)
  return location_df

In [43]:
location_df = place_as_index(train)
location_test_df = place_as_index(test)
train_v2 = pd.merge(train, location_df, on=['latitude', 'longitude'])
test_v2 = pd.merge(test, location_test_df, on=['latitude', 'longitude'])

In [31]:
train_v2.head(20)

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission,No
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994,1
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176,1
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381,1
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286,1
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317,1
5,ID_-0.510_29.290_2019_05,-0.51,29.29,2019,5,0.000294,0.871713,0.000242,0.227656,-13.453690,...,49846.001010,4495.301362,7.556143,0.250228,30.128401,37.007020,-137.388719,21.157820,4.310819,1
6,ID_-0.510_29.290_2019_06,-0.51,29.29,2019,6,-0.000285,0.791956,-0.000226,0.119397,72.198647,...,58804.276563,3537.050659,10.493107,0.240828,4.585368,30.491541,-128.196338,21.721202,4.269334,1
7,ID_-0.510_29.290_2019_07,-0.51,29.29,2019,7,-0.000016,0.976311,-0.000016,0.106799,-100.956055,...,63697.007267,2597.071563,20.533600,0.250168,16.374829,35.840248,-122.250699,20.432005,4.251361,1
8,ID_-0.510_29.290_2019_08,-0.51,29.29,2019,8,-0.000115,0.796941,-0.000101,0.164971,-40.179074,...,67222.556159,2530.851476,24.253928,0.260543,-12.021751,42.688789,-114.296369,22.365602,4.281937,1
9,ID_-0.510_29.290_2019_09,-0.51,29.29,2019,9,0.000056,0.998541,0.000014,0.157726,-100.108744,...,60298.717969,3510.293652,13.813379,0.289295,-66.258392,28.596211,-105.687177,21.084021,4.352933,1


In [50]:
def train_by_lightgbm(df, x_columns, y_column):
  df_train, df_val =train_test_split(df, test_size=0.2)
  train_x = df_train[x_columns]
  train_y = df_train[y_column]

  val_x = df_val[x_columns]
  val_y = df_val[y_column]

  trains = lgb.Dataset(train_x, train_y)
  valids = lgb.Dataset(val_x, val_y)

  params = {
      "objective": "regression",
      "metrics": "mse"
  }
  model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000)
  return model

In [51]:
model = train_by_lightgbm(train_v2, ['year', 'week_no', 'No'], ['emission'])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001900 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 311
[LightGBM] [Info] Number of data points in the train set: 63218, number of used features: 3
[LightGBM] [Info] Start training from score 82.170416


In [52]:
predict = model.predict(test_v2[['year', 'week_no', 'No']])

In [53]:
test['emission'] = predict

In [54]:
test[['ID_LAT_LON_YEAR_WEEK', 'emission']].to_csv('submission.csv', index=False)